In [7]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model

In [ ]:
# Define paths
train_dir = 'D:/Python/python all/shirt-tshirt classification/dataset/train'
val_dir = 'D:/Python/python all/shirt-tshirt classification/dataset/test'

In [ ]:
# Image data generators for training and validation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [ ]:
val_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

In [ ]:
val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

In [ ]:
# Load the VGG16 model without the top layers
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(150, 150, 3))

In [ ]:
# Freeze the layers of the base model
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
# Add new top layers
model = Sequential([
    base_model,
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

In [ ]:
# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=val_generator,
    validation_steps=val_generator.samples // val_generator.batch_size,
    epochs=25
)

In [ ]:
print(train_generator.class_indices)

In [ ]:
# Unfreeze some layers of the base model for fine-tuning
for layer in base_model.layers[-8:]:  # Unfreeze the last 8 layers as an example
    layer.trainable = True

# Recompile the model (necessary after unfreezing layers)
model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

# Fine-tune the model
history_fine = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=val_generator,
    validation_steps=val_generator.samples // val_generator.batch_size,
    epochs=25
)

In [ ]:
# Save the model
model.save('shirt_tshirt_classifier_vgg16.h5')

In [8]:
# Function to classify new images
def classify_image(image_path,model):
    from tensorflow.keras.preprocessing import image
    import numpy as np

    img = image.load_img(image_path, target_size=(150, 150))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0

    prediction = model.predict(img_array)
    if prediction < 0.5:
        return "full_sleeve"
    else:
        return "half_sleeve"

In [ ]:
# Example usage
print(classify_image(r'D:\Python\python all\shirt-tshirt classification\dataset\test\half_sleeve\98.jpg',model))

In [9]:
def load_saved_model(model_path):
    model = load_model(model_path)
    return model

In [10]:
model = load_saved_model(r"D:\Python\python all\shirt-tshirt classification\shirt_tshirt_classifier_vgg16.h5")

In [15]:
# Example usage
print(classify_image(r'D:\Python\python all\shirt-tshirt classification\dataset\test\full_sleeve\352.jpg',model))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 136ms/step
full_sleeve
